#  Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `Hugginh Face`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
"""from langchain_community.llms import Ollama

llm =Ollama(
    model = "phi3"
)"""

'from langchain_community.llms import Ollama\n\nllm =Ollama(\n    model = "phi3"\n)'

## Using Gemini api

In [4]:

from dotenv import load_dotenv
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import os

load_dotenv()

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

llm = llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.2,
    convert_system_message_to_human=True
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm = llm
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm = llm
)


### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm = llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan


In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    llm = llm
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    llm = llm
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    llm = llm
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:

result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer:
**Content Plan: Artificial Intelligence**

**1. Target Audience**

* **Primary Audience:** Business professionals, technology enthusiasts, and individuals interested in the impact of AI on various industries.
* **Secondary Audience:** Students, researchers, and anyone seeking to understand the fundamentals and advancements of AI.

**2. Audience Interests and Pain Points**

**Interests:**

* Latest AI trends and innovations
* Key players and industry leaders
* Real-world applications of AI
* Ethical imp

In [13]:
from IPython.display import Markdown
Markdown(result)

**

**Artificial Intelligence: Transforming Industries and Shaping the Future**

**Introduction**

Artificial Intelligence (AI) has emerged as a transformative force, revolutionizing industries and shaping the very fabric of our society. From self-driving cars to personalized healthcare, AI is leaving an indelible mark on our world. In this blog post, we delve into the latest trends, key players, and ethical implications of AI, exploring its transformative potential and the challenges it presents.

**Key Points**

**Latest Trends in AI**

The field of AI is constantly evolving, with new advancements emerging at a rapid pace. Machine learning, deep learning, and natural language processing are at the forefront of these innovations, enabling computers to learn from data, make predictions, and understand human language. Edge computing and cloud-based AI are also gaining traction, providing real-time data processing and access to vast computing resources.

**Key Players in AI**

Tech giants such as Google, Microsoft, Amazon, and IBM are leading the charge in AI development. However, startups and research institutions are also making significant contributions. Industry leaders across various sectors are embracing AI to enhance their operations and gain a competitive edge.

**Noteworthy News on AI**

Recent breakthroughs in AI have captured the attention of the world. From the development of AI-powered robots that can perform complex tasks to the creation of generative AI models that can create realistic images and text, the field is constantly pushing the boundaries of what is possible.

**Applications of AI**

AI is finding applications in a wide range of industries, transforming the way we live and work. In healthcare, AI is used for disease diagnosis, treatment planning, and drug discovery. In finance, it is employed for fraud detection, risk assessment, and personalized banking. In manufacturing, AI is used for automation, predictive maintenance, and supply chain optimization.

**Ethical Implications of AI**

While AI offers immense potential, it also raises ethical concerns. Job displacement and economic inequality are major concerns, as AI-powered automation could lead to job losses in certain sectors. Bias and discrimination in AI algorithms are also a concern, as they can perpetuate existing societal biases. Privacy concerns and data security are also important considerations, as AI systems rely on vast amounts of data.

**Call to Action**

The future of AI is both exciting and uncertain. As we continue to explore the possibilities and address the challenges, it is essential to engage in informed discussions about the ethical and societal implications of AI. We encourage you to delve deeper into this fascinating field, explore the resources provided, and share your thoughts on the future of AI. Together, we can shape the responsible and transformative use of AI for the benefit of humanity.

In [14]:
topic = "write on Crew.ai"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on write on Crew.ai.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
**Content Plan: Write on Crew.ai**

**1. Target Audience**

* **Demographics:**
    * Writers, content creators, and marketers
    * Individuals seeking to improve their writing skills
    * Professionals looking to enhance their written communication
* **Interests:**
    * Writing techniques and best practices
    * AI-powered writing tools
    * Content strategy and optimization
* **Pain Points:**
    * Writer's block and lack of inspiration
    * Difficulty producing high-quality content efficiently
    * Need for

In [15]:
Markdown(result)

**

**Write on Crew.ai: Empowering Writers with AI-Driven Assistance**

**Introduction**

In the digital age, effective writing is crucial. Writers face challenges like writer's block, time constraints, and the pressure to produce engaging and optimized content. Crew.ai emerges as a game-changer, offering an innovative AI writing assistant that empowers writers to overcome these obstacles and create exceptional content.

**Key Points**

**Section 1: Features and Benefits of Crew.ai**

Crew.ai leverages advanced AI algorithms to provide comprehensive writing assistance. Its robust feature set includes:

* **Grammar and Style Checking:** Crew.ai meticulously checks for grammatical errors, punctuation mistakes, and stylistic inconsistencies, ensuring polished and error-free writing.
* **Plagiarism Detection:** The tool's plagiarism detection capabilities help writers avoid unintentional plagiarism by comparing their work against a vast database of online content.
* **Content Optimization:** Crew.ai analyzes content for readability, SEO optimization, and tone of voice, providing valuable insights to enhance its impact and reach.

**Section 2: Use Cases and Success Stories**

Writers across various industries have experienced firsthand the transformative power of Crew.ai. For instance, a content marketer used the tool to generate engaging blog posts that increased website traffic by 20%. Additionally, a freelance writer utilized Crew.ai's plagiarism detection feature to ensure the originality of their work, leading to increased client satisfaction.

**Section 3: Integration and Pricing**

Crew.ai seamlessly integrates with popular writing platforms such as Google Docs, Microsoft Word, and WordPress, making it accessible to writers in their preferred environments. The tool offers flexible pricing plans tailored to individual needs, ranging from a free basic plan to premium subscriptions with advanced features.

**Call to Action**

If you're a writer seeking to elevate your writing skills, streamline your workflow, and produce exceptional content, Crew.ai is the perfect solution. Visit their website at https://www.crew.ai/ to learn more and start your free trial today. Experience the transformative power of AI-driven writing assistance and unlock your full writing potential.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "Your api key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

#### Openai
```import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
```


#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Hugging Face
```from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="",
    task="text-generation",
)```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).